In [1]:
using HomotopyContinuation, HomologyInferenceWithWeakFeatureSize, DynamicPolynomials
hwfs = HomologyInferecenWithWeakFeatureSize
data_path = escape_string("paper_data/")
threshold = 1e-7

# Order of bottlenecks, change for other orders
k = 2

filename_root = "bots"*string(k)*"_"

# Uncomment if you have re-run the corresponding Bertini script
# data_path = escape_string("paper_data/bertini_inputs/"*str(k)*"_bottlenecks/")
# filename_root = ""

┌ Info: Precompiling HomologyInferenceWithWeakFeatureSize [c51782a9-cb5d-4337-ba91-dcc67d2ae4c4]
└ @ Base loading.jl:1260


"bots2_"

In [2]:
num_vars = 2
@polyvar q[1:num_vars]
x = q[1]
y = q[2]
F = [x^4 - x^2*y^2 + y^4 - 4*x^2 - 2*y^2 - x - 4*y + 1]
singular_points = hwfs.parse_bertini_file(data_path*filename_root*"singular_solutions")
singular_points = [hwfs.butterfly_bertini_convert_solution_point_to_standard_form(point,num_vars,k) for point in singular_points]
nonsingular_points = hwfs.parse_bertini_file(data_path*filename_root*"nonsingular_solutions")
nonsingular_points = [hwfs.butterfly_bertini_convert_solution_point_to_standard_form(point,num_vars,k) for point in nonsingular_points]

nondegenerate_nonsingular = [point for point in nonsingular_points if !hwfs.check_if_solution_is_degenerate(point;threshold=threshold)]
nondegenerate_singular = [point for point in singular_points if !hwfs.check_if_solution_is_degenerate(point;threshold=threshold)]

real_nonsingular = [point for point in nondegenerate_nonsingular if !(abs(imag(hwfs.apply_distance_squared_to_solution(point))) > threshold) ]
putative_real_nonsingular_contributors = [point for point in real_nonsingular if real(hwfs.apply_distance_squared_to_solution(point)) > threshold]
real_singular = [point for point in nondegenerate_singular if !(abs(imag(hwfs.apply_distance_squared_to_solution(point))) > threshold) ]
real_singular_contributors = [point for point in real_singular if real(hwfs.apply_distance_squared_to_solution(point)) > threshold]


distance_system_for_filtering,parameters_for_filtering = hwfs.minimum_distance_start_system(F,q)
start = randn(ComplexF64,num_vars+length(F))
initial_system = subs(distance_system_for_filtering,parameters_for_filtering => start)
initial_solution = solutions(solve(initial_system,start_system = :polyhedral,parameters=:conservative))
distances_nonsingular = [Inf]
if length(putative_real_nonsingular_contributors) > 0
    distances_nonsingular = [hwfs.filter_solution_to_bottleneck(solution,length(q),k,distance_system_for_filtering,start,parameters_for_filtering,initial_solution;threshold=threshold) for solution in putative_real_nonsingular_contributors]
    distances_nonsingular = [dist for dist in distances_nonsingular if dist!=false]
end
distances_singular = [Inf]
if length(real_singular_contributors) > 0 
    distances_singular = [sqrt(hwfs.apply_distance_squared_to_solution(point)) for point in real_singular_contributors]
end




In [3]:
# This number is the total number of solutions computed on B_k(F)
length(singular_points)+length(nonsingular_points)

452

In [4]:
# This counts the solutions which are contained in \Gamma_k(F)
length(singular_points) - length(nondegenerate_singular) + length(nonsingular_points) - length(nondegenerate_nonsingular)

260

In [5]:
# This counts the solutions which are not contained in \Gamma_k(F)
length(nondegenerate_singular)+length(nondegenerate_nonsingular)

192

In [6]:
# The number of non-isolated solutions contributing 
# to the weak feature size
length(nondegenerate_singular)

0

In [7]:
# All non-degenerate non-singular with real distance
println(length(putative_real_nonsingular_contributors)/factorial(k))
# Real algebraic bottlenecks of V(F)
println(length([point for point in putative_real_nonsingular_contributors if hwfs.check_if_vector_is_real(point["circumcenter"];threshold=threshold)])/factorial(k))
# Real algebraic bottlenecks of V(F) \cap R^3
println(length([point for point in putative_real_nonsingular_contributors if hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)])/factorial(k))

96.0
26.0
22.0


In [8]:
# Critical values from non-isolated contributors
minimum(distances_singular)

Inf

In [9]:
# Critical values from isolated geometric k-bottlenecks
minimum(distances_nonsingular)

0.2513222014984109

In [10]:
# Example: This function performs all necessary
# computations and finds the weak feature size 
hwfs.compute_weak_feature_size(F)

Tracking 14336 paths... 100%|████████████████████████████████████████| Time: 0:11:08
  # paths tracked:                  14336
  # non-singular solutions (real):  1888 (0)
  # singular solutions (real):      443 (0)
  # total solutions (real):         2331 (0)


0.25132220149841034